In [1]:
import torchvision
import torch
import torchvision.transforms as transforms
from torchvision import datasets, models

from opacus.grad_sample import GradSampleModule
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, ncx2
from torch.utils.data import TensorDataset, DataLoader

In [2]:
if torch.cuda.is_available() and torch.cuda.device_count() > 0:
    device = torch.device('cuda:0')
else:
    device = torch.device('mps')

In [2]:
from federated_learning.attacks.glir import GaussianDataLoader, SimulatedGradientTracer, DirectGradients, GLiRAttack
from federated_learning.attacks.dp_sgd import recursive_fix
from federated_learning.attacks.utils import analytical_tpr
from federated_learning.model import LeNet_Small_Quant

In [16]:
model = LeNet_Small_Quant()
model = model.eval()
recursive_fix(model)
model.fc = torch.nn.Linear(84, 10)
# for p in model.parameters():
#     p.requires_grad_(False)
model_opacus = GradSampleModule(model, loss_reduction='mean')

In [19]:
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar100_resnet56", pretrained=True)

/Users/vblack/opt/miniconda3/envs/BlockFed/lib/python3.10/site-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/zipball/master" to /Users/vblack/.cache/torch/hub/master.zip
Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/releases/download/resnet/cifar100_resnet56-f2eff4c8.pt" to /Users/vblack/.cache/torch/hub/checkpoints/cifar100_resnet56-f2eff4c8.pt
1

In [20]:
for n, t in model.named_parameters():
    print(n, t.shape, type(t))

conv1.weight torch.Size([16, 3, 3, 3]) <class 'torch.nn.parameter.Parameter'>
bn1.weight torch.Size([16]) <class 'torch.nn.parameter.Parameter'>
bn1.bias torch.Size([16]) <class 'torch.nn.parameter.Parameter'>
layer1.0.conv1.weight torch.Size([16, 16, 3, 3]) <class 'torch.nn.parameter.Parameter'>
layer1.0.bn1.weight torch.Size([16]) <class 'torch.nn.parameter.Parameter'>
layer1.0.bn1.bias torch.Size([16]) <class 'torch.nn.parameter.Parameter'>
layer1.0.conv2.weight torch.Size([16, 16, 3, 3]) <class 'torch.nn.parameter.Parameter'>
layer1.0.bn2.weight torch.Size([16]) <class 'torch.nn.parameter.Parameter'>
layer1.0.bn2.bias torch.Size([16]) <class 'torch.nn.parameter.Parameter'>
layer1.1.conv1.weight torch.Size([16, 16, 3, 3]) <class 'torch.nn.parameter.Parameter'>
layer1.1.bn1.weight torch.Size([16]) <class 'torch.nn.parameter.Parameter'>
layer1.1.bn1.bias torch.Size([16]) <class 'torch.nn.parameter.Parameter'>
layer1.1.conv2.weight torch.Size([16, 16, 3, 3]) <class 'torch.nn.parameter.

In [22]:
def load_cifar10_model():
    model = LeNet_Small_Quant()
    torch.save(model, "CIFAR0.pt")
    model = model.train()
    recursive_fix(model)
    for p in model.parameters():
        p.requires_grad_(False)
    return GradSampleModule(model, loss_reduction='mean'), model

In [6]:
# cifar data
import federated_learning

(X_train, y_train), (X_test, y_test) = federated_learning.load_cifar10(num_users=1, n_class=10, n_samples=2000, rate_unbalance=1.0)
X_train, y_train = X_train[0], y_train[0]

Files already downloaded and verified
Files already downloaded and verified


In [25]:
from federated_learning.attacks.dp_sgd import RandomSubsetDataset
from federated_learning.attacks.glir import ClassificationModelGradients, CheckpointListTracer
from federated_learning.attacks.utils import ListDataLoader

def run_glir_multi_step(params_use, models_use = 5, steps_use = 1):
    """
        params_use: The attack and dataset paramters. See next cell for an example for the structure.
        models_use: Number of models in the name scheme to load. Make sure the corresponding files exist.
        stept_use: How many SGD steps to consider for the attack
    """
    # Load a model and set up the tracing.
    tot_scores_list = []
    criterion = torch.nn.CrossEntropyLoss().to(device)
    for trained_model in range(models_use):
        # Create a model
        opacus_model, weight_load = params_use["model_load"]()
        opacus_model = opacus_model.to(device)
        

        print("Loading model no. ", trained_model)
        # Reset to a past training set that is loaded from a logfile
        tracer = CheckpointListTracer(f"{params_use['modelprefix']}{trained_model}.pt", weight_load)
        #tracer.update_model_to_next_step(opacus_model)

        train_point_idx = tracer.get_used_sample_idx()
        n_in_out_points = len(train_point_idx)
        print("Number of train samples", n_in_out_points)

        # Create loaders for query points, background points, etc.
        # Background dataset: rest of the training data that was not used for test points
        val_dataset = RandomSubsetDataset(params_use["data_train"], subsample_ratio=1.0-(n_in_out_points/len(params_use["data_train"])))
        all_samples = torch.ones(len(params_use["data_train"]))
        all_samples[train_point_idx]=0 # set the used indices to zero.
        val_dataset.sample_idx = torch.where(all_samples==1)[0]
        background_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

        ## Setup a trainpoints loader
        in_train_dataset = RandomSubsetDataset(params_use["data_train"], subsample_ratio=n_in_out_points/len(params_use["data_train"]))
        in_train_dataset.sample_idx = train_point_idx
        base_trainloader = torch.utils.data.DataLoader(in_train_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

        ## Setup loader of test points
        base_testloader = torch.utils.data.DataLoader(params_use["data_test"], batch_size=batch_size, shuffle=False, num_workers=4)

        train_test_loader = ListDataLoader(base_trainloader, base_testloader)

        gradient_comp = ClassificationModelGradients(opacus_model, criterion, cutoff_threshold = params_use["c"], device=device)
        attack = GLiRAttack(background_loader, gradient_comp, tracer, params_use["d"], params_use["n"])
        out_scores = attack.compute_glir_attack_scores_w_loader(train_test_loader, n_load=2*n_in_out_points, n_steps=steps_use)
        labels = torch.cat((torch.ones(n_in_out_points), torch.zeros(n_in_out_points)))
        tot_scores_list.append((out_scores, labels))
    return tot_scores_list

In [24]:
batch_size = 128

cifar_train = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
cifar_test = TensorDataset(torch.tensor(X_test), torch.tensor(y_test))

params_list = {"cifar10": {"data_train": cifar_train, "data_test": cifar_test, "model_load": load_cifar10_model,
                           "modelprefix": "CIFAR", "n_grad_estimations": 49500, "c": float("inf"),
                           "n": 64, "d": 650},
              }

In [26]:
fs_use = np.linspace(0, 1, 100)
ts_rates_all = {}
scores_list = run_glir_multi_step(params_list["cifar10"], models_use=5, steps_use=1)

Loading model no.  0


TypeError: 'LeNet_Small_Quant' object is not subscriptable